In [5]:
from gliner import GLiNER

# Initialize GLiNER with the base model
model = GLiNER.from_pretrained("urchade/gliner_base")
# Sample text for entity prediction
text = """
Der russische Präsident Wladimir Putin hat heute keine neuen Richtlinien verkündet. Die letzten bekannten Äußerungen Putins stammen von einer Fragerunde vor Weihnachten, bei der er über verschiedene Themen wie Syrien, den Krieg gegen die Ukraine und die niedrige Geburtenrate in Russland sprach4
. Bezüglich Joe Biden, Dr. Smith, John Smith und Hillary Clinton liegen keine aktuellen Informationen aus den gegebenen Suchergebnissen vor. Die Suchergebnisse konzentrieren sich hauptsächlich auf Wladimir Putin und enthalten keine Informationen über aktuelle Aktivitäten dieser amerikanischen Politiker oder Personen.
"""

# Labels for entity prediction
# Most GLiNER models should work best when entity types are in lower case or title case
labels = ["Person", "Award", "Date", "Competitions", "Teams"]

# Perform entity prediction
entities = model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

/Users/firaterman/Documents/fer/berserk3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/Users/firaterman/Documents/fer/berserk3/lib/python3.12/site-packages/gliner/model.py:411: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default va

Wladimir Putin => Person
Weihnachten => Date
Joe Biden => Person
Dr. Smith => Person
John Smith => Person
Hillary Clinton => Person
Wladimir Putin => Person


In [9]:
from transformers import DebertaForSequenceClassification

### Deberta SSL


In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=2,  # Binary classification
    ignore_mismatched_sizes=True  # Handle dimension mismatches gracefully
)

# Define a function to classify a single text
def classify_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Get model predictions
    outputs = model(**inputs)
    logits = outputs.logits
    
    # Convert logits to probabilities using softmax
    probabilities = F.softmax(logits, dim=-1)
    
    # Get the predicted class (0 or 1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    # Map class to sentiment
    sentiment = "Positive" if predicted_class == 1 else "Negative"
    
    return sentiment, probabilities

# Test the classifier with individual texts
text_1 = "This movie was fantastic! I absolutely loved it."
text_2 = "This was the worst experience I've ever had."

# Get predictions for individual texts
sentiment_1, probabilities_1 = classify_text(text_1)
sentiment_2, probabilities_2 = classify_text(text_2)

# Print results for individual texts
print(f"Text: {text_1}\nSentiment: {sentiment_1}\nProbabilities: {probabilities_1}")
print()
print(f"Text: {text_2}\nSentiment: {sentiment_2}\nProbabilities: {probabilities_2}")

# Batch processing for multiple texts
texts = [
    "This movie was fantastic! I absolutely loved it.",
    "This was the worst experience I've ever had.",
    "The product is okay, but the delivery was delayed."
]

# Tokenize the texts in a batch
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Get predictions for the batch
outputs = model(**inputs)
logits = outputs.logits
probabilities = F.softmax(logits, dim=-1)
predicted_classes = torch.argmax(probabilities, dim=-1)

# Map class indices to sentiment
sentiments = ["Positive" if cls == 1 else "Negative" for cls in predicted_classes]

# Print results for batch processing
print("\nBatch Processing Results:")
for text, sentiment, probs in zip(texts, sentiments, probabilities):
    print(f"Text: {text}\nSentiment: {sentiment}\nProbabilities: {probs}\n")


Text: This movie was fantastic! I absolutely loved it.
Sentiment: Positive
Probabilities: tensor([[0.4585, 0.5415]], grad_fn=<SoftmaxBackward0>)

Text: This was the worst experience I've ever had.
Sentiment: Positive
Probabilities: tensor([[0.4596, 0.5404]], grad_fn=<SoftmaxBackward0>)

Batch Processing Results:
Text: This movie was fantastic! I absolutely loved it.
Sentiment: Positive
Probabilities: tensor([0.4585, 0.5415], grad_fn=<UnbindBackward0>)

Text: This was the worst experience I've ever had.
Sentiment: Positive
Probabilities: tensor([0.4596, 0.5404], grad_fn=<UnbindBackward0>)

Text: The product is okay, but the delivery was delayed.
Sentiment: Positive
Probabilities: tensor([0.4566, 0.5434], grad_fn=<UnbindBackward0>)



### Deberta offline

In [52]:
import shutil
import os

cache_dir = os.path.expanduser("~/.cache/huggingface")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"Cleared Hugging Face cache: {cache_dir}")

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F

# Load tokenizer and model from local files
local_model_path = "/Users/firaterman/Documents/fer/deberta-v3-small"  # Path to your downloaded model and tokenizer

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    local_model_path,
    num_labels=2,  # Binary classification
    ignore_mismatched_sizes=True  # Handle mismatches if any
)

# Define a function to classify a single text
def classify_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Get model predictions
    outputs = model(**inputs)
    logits = outputs.logits
    
    # Convert logits to probabilities using softmax
    probabilities = F.softmax(logits, dim=-1)
    
    # Get the predicted class (0 or 1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    # Map class to sentiment
    sentiment = "Positive" if predicted_class == 1 else "Negative"
    
    return sentiment, probabilities

# Test the classifier with individual texts
text_1 = "This movie was fantastic! I absolutely loved it."
text_2 = "This was the worst experience I've ever had."

# Get predictions for individual texts
sentiment_1, probabilities_1 = classify_text(text_1)
sentiment_2, probabilities_2 = classify_text(text_2)

# Print results for individual texts
print(f"Text: {text_1}\nSentiment: {sentiment_1}\nProbabilities: {probabilities_1}")
print()
print(f"Text: {text_2}\nSentiment: {sentiment_2}\nProbabilities: {probabilities_2}")

# Batch processing for multiple texts
texts = [
    "This movie was fantastic! I absolutely loved it.",
    "This was the worst experience I've ever had.",
    "The product is okay, but the delivery was delayed."
]

# Tokenize the texts in a batch
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Get predictions for the batch
outputs = model(**inputs)
logits = outputs.logits
probabilities = F.softmax(logits, dim=-1)
predicted_classes = torch.argmax(probabilities, dim=-1)

# Map class indices to sentiment
sentiments = ["Positive" if cls == 1 else "Negative" for cls in predicted_classes]

# Print results for batch processing
print("\nBatch Processing Results:")
for text, sentiment, probs in zip(texts, sentiments, probabilities):
    print(f"Text: {text}\nSentiment: {sentiment}\nProbabilities: {probs}\n")


Text: This movie was fantastic! I absolutely loved it.
Sentiment: Positive
Probabilities: tensor([[0.4917, 0.5083]], grad_fn=<SoftmaxBackward0>)

Text: This was the worst experience I've ever had.
Sentiment: Positive
Probabilities: tensor([[0.4917, 0.5083]], grad_fn=<SoftmaxBackward0>)

Batch Processing Results:
Text: This movie was fantastic! I absolutely loved it.
Sentiment: Positive
Probabilities: tensor([0.4917, 0.5083], grad_fn=<UnbindBackward0>)

Text: This was the worst experience I've ever had.
Sentiment: Positive
Probabilities: tensor([0.4917, 0.5083], grad_fn=<UnbindBackward0>)

Text: The product is okay, but the delivery was delayed.
Sentiment: Positive
Probabilities: tensor([0.4903, 0.5097], grad_fn=<UnbindBackward0>)



### Gliner

In [20]:
local_gliner_path = "/Users/firaterman/Documents/fer/gliner_small-v2.1" 
print("file in", os.listdir(local_gliner_path))

local_gliner_path = "/Users/firaterman/Documents/fer/gliner_small-v2.1" 
print("file in", os.listdir(local_gliner_path))

file in ['gitattributes', 'gliner_config.json', 'README.md', '.gitattributes', 'pytorch_model.bin', '.git']


In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import torch
import torch.nn.functional as F

# Paths to local DeBERTa and Gliner files
deberta_path = "/Users/firaterman/Documents/fer/deberta-v3-small"
gliner_weights_path = "/Users/firaterman/Documents/fer/gliner_small-v2.1/pytorch_model.bin"

# Load tokenizer and configuration offline
tokenizer = AutoTokenizer.from_pretrained(deberta_path)
config = AutoConfig.from_pretrained(deberta_path)

# Update configuration for Gliner-specific task
config.num_labels = 3  # Adjust based on Gliner task requirements

# Load Gliner weights into DeBERTa model
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=None,  # Not using a pretrained name
    config=config,
    state_dict=torch.load(gliner_weights_path, map_location=torch.device("cpu"))
)

# Define an inference function
def gliner_predict(text):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits
    
    # Compute probabilities and predicted class
    probabilities = F.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    # Map predicted class to labels
    labels = ["Location", "Person", "City"]  # Update based on Gliner task
    prediction = labels[predicted_class]
    return prediction, probabilities

# Example prediction
example_text = "Jean is present in Paris"
prediction, probabilities = gliner_predict(example_text)
print(f"Prediction: {prediction}, Probabilities: {probabilities}")


/var/folders/dr/83m5p3m94s7gmtrc3_97lxvc0000gn/T/ipykernel_94559/3279719854.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load(gliner_weights_path, m

Prediction: City, Probabilities: tensor([[0.3172, 0.3323, 0.3505]], grad_fn=<SoftmaxBackward0>)


In [28]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification
import torch

# Paths to local DeBERTa and Gliner weights
deberta_path = "/Users/firaterman/Documents/fer/deberta-v3-small"
gliner_weights_path = "/Users/firaterman/Documents/fer/gliner_small-v2.1/pytorch_model.bin"

# Load tokenizer and configuration offline
tokenizer = AutoTokenizer.from_pretrained(deberta_path)
config = AutoConfig.from_pretrained(deberta_path)

# Update configuration for token classification
config.num_labels = 7  # Number of NER labels (O, B-PER, I-PER, B-LOC, I-LOC, B-DATE, I-DATE)
config.id2label = {0: "O", 1: "B-PER", 2: "I-PER", 3: "B-LOC", 4: "I-LOC", 5: "B-DATE", 6: "I-DATE"}
config.label2id = {label: id for id, label in config.id2label.items()}

# Load Gliner weights into DeBERTa model for token classification
model = AutoModelForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=None,
    config=config,
    state_dict=torch.load(gliner_weights_path, map_location=torch.device("cpu"))
)

# Function for NER predictions
def gliner_ner_predict(text):
    # Tokenize the input
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    input_ids = tokens["input_ids"]
    
    # Perform inference
    outputs = model(**tokens)
    logits = outputs.logits
    
    # Convert logits to predicted labels
    predictions = torch.argmax(logits, dim=-1).squeeze().tolist()
    
    # Decode tokens and align predictions
    tokens_decoded = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
    entities = []
    for token, label_id in zip(tokens_decoded, predictions):
        label = config.id2label[label_id]
        if label != "O":  # Skip non-entity tokens
            entities.append((token, label))
    
    return entities

# Example prediction
example_text = "John Doe visited New York on January 15, 2023."
entities = gliner_ner_predict(example_text)

# Print results
for entity, label in entities:
    print(f"Entity: {entity}, Label: {label}")


/var/folders/dr/83m5p3m94s7gmtrc3_97lxvc0000gn/T/ipykernel_94559/4273152854.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load(gliner_weights_path, m

Entity: [CLS], Label: I-PER
Entity: ▁John, Label: B-DATE
Entity: ▁Doe, Label: I-LOC
Entity: ▁visited, Label: B-DATE
Entity: ▁New, Label: B-DATE
Entity: ▁York, Label: B-DATE
Entity: ▁on, Label: B-DATE
Entity: ▁January, Label: B-LOC
Entity: ▁15, Label: B-DATE
Entity: ,, Label: B-LOC
Entity: ▁2023, Label: B-LOC
Entity: ., Label: B-DATE
Entity: [SEP], Label: I-PER


In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import torch
import torch.nn.functional as F

# Paths to local DeBERTa and GLiNER model files
deberta_path = "/Users/firaterman/Documents/fer/deberta-v3-small"  # Path to DeBERTa files
gliner_weights_path = "/Users/firaterman/Documents/fer/gliner_small-v2.1/pytorch_model.bin"  # Path to GLiNER weights

# Load tokenizer and configuration offline
tokenizer = AutoTokenizer.from_pretrained(deberta_path)
config = AutoConfig.from_pretrained(deberta_path)

# Update configuration for GLiNER-specific task
config.num_labels = 3  # Number of labels for GLiNER task (update based on your labels)

# Load GLiNER weights into DeBERTa model
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=None,  # No internet usage
    config=config,
    state_dict=torch.load(gliner_weights_path, map_location=torch.device("cpu"))
)

# Define a prediction function for GLiNER with DeBERTa
def gliner_predict(text, labels, threshold=0.5):
    """
    Predict entities in text using the GLiNER model.
    
    Args:
        text (str): The input text for prediction.
        labels (list): List of possible labels.
        threshold (float): Minimum probability for classification.
        
    Returns:
        tuple: Predicted label and probabilities.
    """
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits
    
    # Compute probabilities for each label
    probabilities = F.softmax(logits, dim=-1)
    
    # Get the most probable class and its label
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    prediction = labels[predicted_class]
    
    return prediction, probabilities

# Define possible labels for the GLiNER task
labels = ["Location", "Person", "City"]  # Update based on GLiNER's specific task requirements

# Example usage
example_text = "Cristiano Ronaldo was born in Madeira"
prediction, probabilities = gliner_predict(example_text, labels)

# Output the result
print(f"Prediction: {prediction}")
print(f"Probabilities: {probabilities}")


/var/folders/dr/83m5p3m94s7gmtrc3_97lxvc0000gn/T/ipykernel_94559/2365459257.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load(gliner_weights_path, m

Prediction: Location
Probabilities: tensor([[0.3950, 0.2735, 0.3314]], grad_fn=<SoftmaxBackward0>)


In [35]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import torch
import torch.nn.functional as F

# Paths to local DeBERTa and GLiNER files
deberta_path = "/Users/firaterman/Documents/fer/deberta-v3-small"  # Path to DeBERTa files
gliner_weights_path = "/Users/firaterman/Documents/fer/gliner_small-v2.1/pytorch_model.bin"  # Path to GLiNER weights

# Load tokenizer and configuration offline
tokenizer = AutoTokenizer.from_pretrained(deberta_path)
config = AutoConfig.from_pretrained(deberta_path)

# Update configuration for GLiNER-specific task
config.num_labels = 2  # Number of entity labels (update as needed for your task)

# Load GLiNER weights into DeBERTa model for token classification
model = AutoModelForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=None,
    config=config,
    state_dict=torch.load(gliner_weights_path, map_location=torch.device("cpu"))
)

# Define labels for the GLiNER task
labels = ["Person", "Location"]  # Update as per GLiNER's label set


def predict_entities(text, labels, threshold=0.5):
    """
    Predict named entities in the text using GLiNER.
    
    Args:
        text (str): Input text to process.
        labels (list): List of entity labels (first label is usually "O" for non-entities).
        threshold (float): Minimum probability threshold for classification.

    Returns:
        list: List of entities with their text and labels.
    """
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, return_offsets_mapping=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    offset_mapping = inputs.pop("offset_mapping")[0].tolist()

    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=-1)

    # Extract predictions for each token
    predictions = torch.argmax(probabilities, dim=-1).squeeze().tolist()

    # Map predictions back to original text
    entities = []
    for i, pred in enumerate(predictions):
        if pred != 0:  # Ignore "O" (non-entity tokens)
            entity_label = labels[pred]
            start, end = offset_mapping[i]
            entity_text = text[start:end]
            entities.append({"text": entity_text, "label": entity_label})

    return entities


# Example text
example_text = """
Cristiano Ronaldo dos Santos Aveiro (born 5 February 1985) is a Portuguese professional footballer who plays for Al Nassr.
"""

# Perform entity prediction
entities = predict_entities(example_text, labels)

# Display results
for entity in entities:
    print(f"{entity['text']} => {entity['label']}")


/var/folders/dr/83m5p3m94s7gmtrc3_97lxvc0000gn/T/ipykernel_94559/4116677029.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load(gliner_weights_path, m

Cristiano => Location
 do => Location
s => Location
iro => Location
born => Location
 5 => Location
 1985 => Location
 is => Location
 a => Location
 professional => Location
 footballer => Location
 plays => Location
 for => Location
 Al => Location
 Nass => Location
. => Location
 => Location


In [50]:
import torch
from pathlib import Path
from gliner import GLiNER
import json
from types import SimpleNamespace

def load_local_gliner(gliner_path, deberta_path):
    gliner_path = Path(gliner_path)
    deberta_path = Path(deberta_path)

    # Load GLiNER config
    with open(gliner_path / "gliner_config.json", 'r') as f:
        config_dict = json.load(f)
    
    # Convert dictionary to namespace
    config = SimpleNamespace(**config_dict)
    
    # Point to local DeBERTa path
    config.model_name = str(deberta_path)

    # Initialize GLiNER model with config namespace
    model = GLiNER(config)
    
    # Load GLiNER weights
    state_dict = torch.load(
        gliner_path / "pytorch_model.bin",
        map_location=torch.device("cpu")
    )
    model.load_state_dict(state_dict, strict=False)
    model.to("cpu")
    
    return model

# Paths to local models
deberta_path = "/Users/firaterman/Documents/fer/deberta-v3-small"
gliner_path = "/Users/firaterman/Documents/fer/gliner_small-v2.1"

# Load model
model = load_local_gliner(gliner_path, deberta_path)

# Sample text
text = """
Obama is the president of the USA and he will visit France with Trump
"""

# Define labels
labels = ["Person", "Date", "Country", "Location"]

# Predict entities
entities = model.predict_entities(text, labels, threshold=0.5)

# Display results
for entity in entities:
    print(entity["text"], "=>", entity["label"])

/var/folders/dr/83m5p3m94s7gmtrc3_97lxvc0000gn/T/ipykernel_94559/1704549477.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


Obama => Person
USA => Country
France => Country
Trump => Person


In [5]:
import torch
from pathlib import Path
from gliner import GLiNER
import json
from types import SimpleNamespace

def load_local_gliner(gliner_path, deberta_path):
    gliner_path = Path(gliner_path)
    deberta_path = Path(deberta_path)

    # Load GLiNER config
    with open(gliner_path / "gliner_config.json", 'r') as f:
        config_dict = json.load(f)
    
    # Convert dictionary to namespace
    config = SimpleNamespace(**config_dict)
    
    # Point to local DeBERTa path
    config.model_name = str(deberta_path)

    # Initialize GLiNER model with config namespace
    model = GLiNER(config)
    
    # Load GLiNER weights
    state_dict = torch.load(
        gliner_path / "pytorch_model.bin",
        map_location=torch.device("cpu")
    )
    model.load_state_dict(state_dict, strict=False)
    model.to("cpu")
    
    return model

# Paths to local models
deberta_path = "/Users/firaterman/Documents/fer/mdberta_v3_base"
gliner_path = "/Users/firaterman/Documents/fer/gliner_multi_v2.1"

# Load model
model = load_local_gliner(gliner_path, deberta_path)

# Sample text
text = "Obama is the president of the USA and he will visit France with Trump. The employees were there for him. DMC is specialized in making cars. There is also Harry Potter present there. Microsoft is also doing some updates. The man is sick. Jean is absent. The firm is generating money"

# Define labels
labels = ["Person", "Date", "Country", "Location"]

# Predict entities
entities = model.predict_entities(text, labels, threshold=0.5)

# Display results
for entity in entities:
    print(entity["text"], "=>", entity["label"])

/var/folders/dr/83m5p3m94s7gmtrc3_97lxvc0000gn/T/ipykernel_28424/4103473976.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


Obama => Person
president => Person
USA => Location
France => Location
Trump => Person
employees => Person
DMC => Person
Harry Potter => Person
man => Person
Jean => Person


In [27]:
import torch
from pathlib import Path
from gliner import GLiNER
import json
from types import SimpleNamespace

def load_local_gliner(gliner_path, deberta_path):
   gliner_path = Path(gliner_path)
   deberta_path = Path(deberta_path)
   
   with open(gliner_path / "gliner_config.json", 'r') as f:
       config_dict = json.load(f)
   config = SimpleNamespace(**config_dict)
   
   config.model_name = str(deberta_path)

   model = GLiNER(config)
   state_dict = torch.load(
       gliner_path / "pytorch_model.bin",
       map_location=torch.device("cpu")
   )
   model.load_state_dict(state_dict, strict=True)
   model.to("cpu")
   return model

# Paths
deberta_path = "/Users/firaterman/Documents/fer/mdberta_v3_base"
gliner_path = "/Users/firaterman/Documents/fer/gliner_multi_v2.1"

# Load and predict
model = load_local_gliner(gliner_path, deberta_path)

text = "Obama is the president of the USA and he will visit France with Trump. The employees were there for him. DMC is specialized in making cars. There is also Harry Potter present there. Microsoft is also doing some updates. The man is sick. Jean is absent. The firm is generating money"
# text= "The firm offers accountancy, taxation, financial consultancy and insolvency services to foreign and local clients"

# Define labels
labels = ["Person", "organization", "company"]
entities = model.predict_entities(text, labels, threshold=0.1)

print(entities)
for entity in entities:
   if entity["score"]>0.5:
      print(entity["text"], "=>", entity["label"])

/var/folders/dr/83m5p3m94s7gmtrc3_97lxvc0000gn/T/ipykernel_28424/3943298557.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


[{'start': 0, 'end': 5, 'text': 'Obama', 'label': 'Person', 'score': 0.9598107933998108}, {'start': 13, 'end': 22, 'text': 'president', 'label': 'Person', 'score': 0.23888185620307922}, {'start': 30, 'end': 33, 'text': 'USA', 'label': 'organization', 'score': 0.1632390171289444}, {'start': 64, 'end': 69, 'text': 'Trump', 'label': 'Person', 'score': 0.9008338451385498}, {'start': 105, 'end': 108, 'text': 'DMC', 'label': 'organization', 'score': 0.6567029356956482}, {'start': 154, 'end': 166, 'text': 'Harry Potter', 'label': 'Person', 'score': 0.6932865381240845}, {'start': 182, 'end': 191, 'text': 'Microsoft', 'label': 'organization', 'score': 0.6205271482467651}, {'start': 224, 'end': 227, 'text': 'man', 'label': 'Person', 'score': 0.1237051859498024}, {'start': 237, 'end': 241, 'text': 'Jean', 'label': 'Person', 'score': 0.8641250729560852}, {'start': 257, 'end': 261, 'text': 'firm', 'label': 'organization', 'score': 0.11162350326776505}]
Obama => Person
Trump => Person
DMC => organiz